# ***YOLOv12 + R-ELAN + PANet***

# YOLOv12m + R-ELAN + PANET
**Hyperparameter Tuning Optuna + Final Training 500 Epoch**  
**Model:** `yolo12m.yaml`
**Dataset:** Dental Caries v13 (7 kelas)  
**Objective:** Combined Score = 0.5×mAP50-95 + 0.3×Recall + 0.2×Precision  

## 1. Install Ultralytics Terbaru & Import Library

In [1]:
!pip install -q ultralytics optuna pandas roboflow PyYAML --upgrade

import optuna
from ultralytics import YOLO
from pathlib import Path
from roboflow import Roboflow
import yaml
import json
import pandas as pd

## 2. Pengaturan Global

In [2]:
VARIASI_NAME     = "yolo12m_r-elan_panet"
VERSION_NUM      = 13
EPOCHS_SEARCH    = 20
EPOCHS_FINAL     = 500
BATCH_CANDIDATES = [16, 24, 32, 48, 64, 96]

OPTUNA_ROOT      = Path("runs/optuna")
VARIASI_DIR      = OPTUNA_ROOT / VARIASI_NAME
VARIASI_DIR.mkdir(parents=True, exist_ok=True)

DB_PATH   = VARIASI_DIR / "optuna_study.db"
JSON_PATH = VARIASI_DIR / "best_hyperparameters.json"
CSV_PATH  = VARIASI_DIR / "optuna_trials_log.csv"

MODEL_NAME = "yolo12m.yaml"

## 3. Download Dataset Dental Caries

In [3]:
API_KEY     = "QOd5ldAdjiaehHn5m6WC"
WORKSPACE   = "dentalogic8"
PROJECT_ID  = "dental-caries-7kttb"

rf = Roboflow(api_key=API_KEY)
project = rf.workspace(WORKSPACE).project(PROJECT_ID)
dataset = project.version(VERSION_NUM).download("yolov12")

DATASET_DIR = Path(f"dental-caries-{VERSION_NUM}")
DATA_YAML   = DATASET_DIR / "data.yaml"

if not DATA_YAML.exists():
    raise FileNotFoundError("data.yaml tidak ditemukan!")

print(f"Dataset v{VERSION_NUM} siap → {DATA_YAML}")

loading Roboflow workspace...
loading Roboflow project...
Dataset v13 siap → dental-caries-13/data.yaml


## 4. Buat CSV Log untuk Analisis Trial

In [4]:
def init_csv():
    if not CSV_PATH.exists():
        pd.DataFrame(columns=[
            "Trial","Batch","Epochs","lr0","lrf","weight_decay",
            "box","cls","dfl","mAP@50","mAP@50-95","Precision","Recall",
            "Combined_Score","Status"
        ]).to_csv(CSV_PATH, index=False)
        print(f"CSV log dibuat → {CSV_PATH}")
init_csv()

CSV log dibuat → runs/optuna/yolo12m_r-elan_panet/optuna_trials_log.csv


## 5. Combined Score — Objective Utama Optuna  
Didesain khusus untuk dental caries detection:  
- **50%** mAP50-95 → akurasi deteksi keseluruhan  
- **30%** Recall → sangat penting (tidak boleh ada karies yang terlewat)  
- **20%** Precision → tetap kontrol false positive

In [5]:
def calculate_combined_score(m):
    map5095   = m.get("metrics/mAP50-95(B)", 0.0)
    recall    = m.get("metrics/recall(B)", 0.0)
    precision = m.get("metrics/precision(B)", 0.0)
    return 0.5 * map5095 + 0.3 * recall + 0.2 * precision, map5095, recall, precision

## 6. Optuna Hyperparameter Search  
Menggunakan `yolo12m.yaml`

In [ ]:
study = optuna.create_study(
    study_name=f"dental_yolo12m_official",
    direction="maximize",
    sampler=optuna.samplers.TPESampler(seed=42),
    storage=f"sqlite:///{DB_PATH}",
    load_if_exists=True
)

def objective(trial):
    batch = trial.suggest_categorical("batch", BATCH_CANDIDATES)
    lr0   = trial.suggest_float("lr0", 1e-4, 0.1, log=True)
    lrf   = trial.suggest_float("lrf", 0.005, 0.2, log=True)
    wd    = trial.suggest_float("weight_decay", 1e-5, 1e-3, log=True)
    box   = trial.suggest_float("box", 7.5, 25.0)
    cls   = trial.suggest_float("cls", 0.3, 1.5)
    dfl   = trial.suggest_float("dfl", 0.5, 4.0)

    model = YOLO(MODEL_NAME)

    try:
        results = model.train(
            data=str(DATA_YAML),
            imgsz=640,
            epochs=EPOCHS_SEARCH,
            batch=batch,
            device=1,
            project=str(VARIASI_DIR),
            name=f"trial_{trial.number:04d}",
            exist_ok=True,
            pretrained=False,
            optimizer="AdamW",
            lr0=lr0, lrf=lrf, weight_decay=wd,
            box=box, cls=cls, dfl=dfl,
            patience=10,
            cache="disk",
            verbose=False,
            plots=False,
            close_mosaic=5,
            amp=True
        )
        score, map5095, recall, precision = calculate_combined_score(results.results_dict)
        map50 = results.results_dict.get("metrics/mAP50(B)", 0.0)
        status = "Completed"
    except Exception as e:
        print(f"Trial {trial.number} gagal → {e}")
        score = map5095 = recall = precision = map50 = 0.0
        status = "Failed"

    pd.DataFrame([{
        "Trial": trial.number, "Batch": batch, "Epochs": EPOCHS_SEARCH,
        "lr0": round(lr0,8), "lrf": round(lrf,6), "weight_decay": wd,
        "box": round(box,4), "cls": round(cls,4), "dfl": round(dfl,4),
        "mAP@50": round(map50,5), "mAP@50-95": round(map5095,5),
        "Precision": round(precision,5), "Recall": round(recall,5),
        "Combined_Score": round(score,6), "Status": status
    }]).to_csv(CSV_PATH, mode='a', header=False, index=False)

    print(f"Trial {trial.number:3d} │ B{batch:3d} │ mAP50-95 {map5095:.5f} │ Combined {score:.6f}")
    return score

print(f"{'='*95}")
print(f"OPTUNA SEARCH: YOLOv12m OFFICIAL (R-ELAN + PANet + A2C2f)")
print(f"Model → {MODEL_NAME} (100% resmi Ultralytics)")
print(f"Trial: {EPOCHS_SEARCH} epoch │ Final: {EPOCHS_FINAL} epoch")
print(f"{'='*95}\n")

study.optimize(objective, n_trials=20)

[I 2025-11-22 14:10:03,285] A new study created in RDB with name: dental_yolo12m_official


OPTUNA SEARCH: YOLOv12m OFFICIAL (R-ELAN + PANet + A2C2f)
Model → yolo12m.yaml (100% resmi Ultralytics)
Trial: 20 epoch │ Final: 500 epoch

Ultralytics 8.3.230 🚀 Python-3.10.12 torch-2.9.1+cu128 CUDA:1 (NVIDIA H200, 143167MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=24, bgr=0.0, box=19.891270111430796, cache=disk, cfg=None, classes=None, close_mosaic=5, cls=0.32470139315496294, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dental-caries-13/data.yaml, degrees=0.0, deterministic=True, device=1, dfl=3.89468448256698, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.00014936568554617635, lrf=0.12207764786954153, mask_ratio=4, max_det=300, mixup=0.0, mode=tr

## 7. Simpan Hyperparameter Terbaik

In [ ]:
with open(JSON_PATH, "w") as f:
    json.dump(study.best_params, f, indent=4)

print(f"\nOPTIMASI SELESAI!")
print(f"Best Combined Score : {study.best_value:.6f}")
print(f"Best Trial          : {study.best_trial.number}")
print(f"Best params saved   → {JSON_PATH}")

## 8. Final Training → yolo12m.yaml + Best Hyperparameter

In [ ]:
!pip install -q ultralytics --upgrade
from ultralytics import YOLO
import json

with open(JSON_PATH) as f:
    best = json.load(f)

print("Best Hyperparameters:")
for k, v in best.items():
    print(f"  {k:12} → {v}")

model = YOLO(MODEL_NAME)

model.train(
    data=str(DATA_YAML),
    imgsz=640,
    epochs=EPOCHS_FINAL,
    batch=best["batch"],
    device=0,
    project="runs/final",
    name="yolo12m_official_FINAL",
    exist_ok=True,
    pretrained=False,
    optimizer="AdamW",
    patience=50,
    cache="disk",
    plots=True,
    save=True,
    close_mosaic=10,
    lr0=best["lr0"],
    lrf=best["lrf"],
    weight_decay=best["weight_decay"],
    box=best["box"],
    cls=best["cls"],
    dfl=best["dfl"],
)

print(f"\nFINAL TRAINING SELESAI! Model terbaik:")
print(f"runs/final/yolo12m_official_FINAL/weights/best.pt")